# Newton's Method

In [ ]:
using Plots

In [ ]:
default(lw=2,markersize = 6,
    xtickfont=font(12), ytickfont=font(12), 
    guidefont=font(14), legendfont=font(12), 
    titlefont=font(12))

Code for Newton's method for function `f` with derivative `df`, and initial guess `p0`

In [ ]:
function newton(f, df, p0, n_max, rel_tol; verbose = true)
    
    converged = false;
    p = p0;
    p_old = p0;

    for i in 1:n_max

        p = p_old - f(p_old)/df(p_old);
        
        if verbose
            println(" $i: p = $(p), |f(p)| = $(abs(f(p)))")
        end

        if (i>1)
            if abs(p-p_old)/abs(p)< rel_tol
                converged = true;
                break
            end
        end

        p_old = p;

    end
    
    if !converged
        println("ERROR: Did not converge after $n_max iterations")
    end

    return p
    
end

Code for bisection method `f` on interval `[a,b]`

In [ ]:
"""
    bisection(f, a, b, n_max, tol; verbose = true)

Find a root of function `f` in the interval [a,b] using the bisection method.

# Arguments
- `f`: Function for which to find the root
- `a`: Left endpoint of the initial interval (must satisfy f(a)*f(b) < 0)
- `b`: Right endpoint of the initial interval (must satisfy f(a)*f(b) < 0)
- `n_max`: Maximum number of iterations allowed
- `tol`: Tolerance for convergence (algorithm stops when interval width < 2*tol)
- `verbose`: Optional keyword argument. If true, prints iteration information (default: true)

# Returns
- `p`: Approximate root of the function

# Method
The bisection method works by:
1. Computing the midpoint p = (a+b)/2 of the current interval [a,b]
2. Evaluating f(p) and determining which subinterval [a,p] or [p,b] contains the root
3. Replacing the interval with the subinterval that contains the root
4. Repeating until convergence or maximum iterations reached

# Convergence
The algorithm converges when either:
- f(p) = 0 (exact root found)
- The interval width (b-a)/2 < tol

# Example
```julia
f(x) = x^3 + 4*x^2 - 10
root = bisection(f, 1, 2, 100, 1e-8)
```

# Notes
- Requires that f(a) and f(b) have opposite signs (Intermediate Value Theorem)
- Convergence is guaranteed but may be slow compared to other methods
- Error decreases by factor of 2 each iteration: |p_n - p| ≤ (b-a)/2^n
"""
function bisection(f, a, b, n_max, tol; verbose = true)
    
    converged = false;
    p = 0;
    for i in 1:n_max

        p = 0.5 * (a+b); # compute the midpoint 
        
        # print current iterate information to screen 
        if verbose
            println(" $i: a = $(round(a,digits=8)), b = $(round(b,digits=8)), p = $(round(p,digits=8)), |f(p)|  = $(round(abs(f(p)),digits=8))")
        end

        # determine if the root is in the left or right interval
        if ( f(a) * f(p)<=0)
            b = p; # root is interval [a,p]
        else
            a = p # root is in interval [p,b]
        end
        if(f(p)==0)
            converged = true;
            break
        end
        
        # test for convergence
        if .5*(b-a)< tol
            converged = true;
            break
        end
    end
    
    if !converged
        println("ERROR: Did not converge after $n_max iterations")
    end

    return p # return midpoint guess
    
end

## Example 
Compute $\sqrt{2}$ by solving $f(x) = 0$ with
$$
f(x) = x^2-2.
$$

In [ ]:
f = x-> x^2 - 2;
df = x->2*x;
p0 = 1;
rel_tol = 1e-8;
n_max = 100;

p = newton(f, df, p0, n_max, rel_tol);

In [ ]:
@show sqrt(2);

### Comparison with Bisection

In [ ]:
a = 1;
b = 2;
rel_tol = 1e-8;
n_max = 100;

p = bisection(f,a, b, n_max, rel_tol);

When Newton works, it is *much* faster than bisection.  But:
* It requires $f$ to be at least differentiable and theorems for convergence require $C^2$
* The derviative needs to be coded up

### Visualization

In [ ]:
n_max = 5;
p_vals = zeros(n_max+1);
p0 = 1;

p = p0;
p_vals[1] = p;
for i in 1:n_max
    p = p - f(p)/df(p);
    p_vals[i+1] = p;

end

xx = LinRange(0,2,100);

ff = f.(xx);
plot(xx, ff, label="f", legend=:topleft)
plot!(xx, 0 *xx,label="", color=:black)

anim = @animate for i=1:n_max+1
    
    p = p_vals[i];
    
    plot!([p, p], [0, f(p)], label="", color=:red, ls=:dash)
    plot!([p, p- f(p)/df(p)],[f(p), 0],label="", color=:red)
    xlims!(0.9,1.6)
    ylims!(-1, 0.5)
    
    xlabel!("x");
    ylabel!("y")
    
    title!(string("n = $i"))
end;



In [ ]:
gif(anim,  fps = 1)

## Example
Find a root of
$$
f(x) = \cos(x) - x
$$

In [ ]:
f = x-> cos(x) -x;
df = x->-sin(x) -1;
p0 = 0;
rel_tol = 1e-8;
n_max = 100;

p = newton(f, df, p0, n_max, rel_tol);

In [ ]:
xx = LinRange(0,π/2,100);
plot(xx, f.(xx), label="f")
plot!(xx, 0*xx, label="")
scatter!([p], [f(p)],label="Root")

# Automatic Differentiation

Automatic differentiation often lets us avoid having to compute a derivative by hand, before running Newton's method.

In [ ]:
using ForwardDiff # automatic differentiation package

Automatically differentiate `f` at `x`:

In [ ]:
df_auto = x-> ForwardDiff.derivative(f,x)

In [ ]:
xx = LinRange(0,π/2,100);
plot(xx, df.(xx), label="f' by hand")
plot!(xx, df_auto.(xx), label="f' by AD")


Apply Newton with a derivative created through automatic differentiation

In [ ]:
p = newton(f, df_auto, p0, n_max, rel_tol);

Use automatic differentiation, if possible